In [ ]:
#Packages
import pingouin as pg
from scipy.stats import shapiro
import pandas as pd

In [ ]:
def check_sphericity(df: pd.DataFrame) -> None:
    """Performs Mauchly’s test of sphericity on different within-subject factors."""
    print("\nSphericity test for attack_type:")
    print(pg.sphericity(df, dv='image_quality', subject='image_id', within=['attack_type']))
    
    print("\nSphericity test for target_label:")
    print(pg.sphericity(df, dv='image_quality', subject='image_id', within=['target_label']))
    
    print("\nSphericity test for interaction (attack_type × target_label):")
    print(pg.sphericity(df, dv='image_quality', subject='image_id', within=['attack_type', 'target_label']))

In [ ]:
def compute_residuals(df: pd.DataFrame) -> pd.Series:
    """Computes group mean and residuals for normality check."""
    df['group_mean'] = df.groupby(['image_id', 'attack_type', 'target_label'])['image_quality'].transform('mean')
    df['residual'] = df['image_quality'] - df['group_mean']
    return df['residual']

In [ ]:
def check_normality(residuals: pd.Series) -> None:
    """Performs Shapiro–Wilk test for normality of residuals."""
    stat, p = shapiro(residuals)
    print(f"\nShapiro-Wilk test for residuals: W={stat:.4f}, p={p:.4f}")
    if p > 0.05:
        print("Residuals appear normally distributed.")
    else:
        print("Residuals may not be normally distributed.")